### SoccerNet Object Tracking
#### 02456 Deep learning 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import time
import yaml
import re
import seaborn as sn
from IPython.display import Video, Image
from glob import glob
from io import TextIOWrapper
import collections
import zipfile
import csv
import cv2

##### Data downloader

In [2]:
# import SoccerNet
# from SoccerNet.Downloader import SoccerNetDownloader
# mySoccerNetDownloader=SoccerNetDownloader(LocalDirectory="path/to/SoccerNet")
# mySoccerNetDownloader.downloadDataTask(task="tracking", split=["train", "test", "challenge"])

##### Function to visualize boxes

In [3]:
def get_np_array_from_tar_object(tar_extractfl):
     '''converts a buffer from a tar file in np.array'''
     return np.asarray(bytearray(tar_extractfl.read()), dtype=np.uint8)

def collect(fn):
    archive = zipfile.ZipFile(fn, 'r')
    annotated_images = []

    annotation_files = []
    for f in archive.namelist():
        if "det/det.txt" in f:
            annotation_files.append(f)

    image_annotations = collections.defaultdict(list)

    for af in annotation_files:
        st, sample,_,__ = af.split("/")
        imgpath = "%s/%s/img1/%06d.jpg"
        for row in csv.reader(TextIOWrapper(archive.open(af), 'utf-8') ):
            frame, _, x, y, w, h = [int(x) for x in row[:6]]
            ifn = imgpath%(st,sample, frame)
            image_annotations[ifn].append((x,y,w,h))

    for k in image_annotations.keys():
        img = cv2.imdecode(get_np_array_from_tar_object(archive.open(k)), 1 )
        for box in image_annotations[k]:
            x,y,w,h = box
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0), 2)

        img = cv2.resize(img, (0,0), fx=0.75, fy=0.75)
        cv2.imshow('image',img)
        cv2.waitKey(100)

In [4]:
# # Visualize
# train = collect("path/to/SoccerNet/tracking/train.zip")
# cv2.destroyAllWindows()

##### Test YOLOv7 with SoccerNet

In [2]:
yolo_base = 'yolov7'
sn_track_base = 'path/to/SoccerNet/tracking'

# ４ types of labels are used.
labels = ['ball', 'player', 'referee', 'goalkeepers']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}

In [7]:
# Of the 57 half , 50 are used for training and 7 for validation.
#trn_dirs = sorted(glob(f'{sn_track_base}/train/SNMOT*'))
#tst_dirs = sorted(glob(f'{sn_track_base}/test/SNMOT*'))
trn_dirs = ["C:/Users/Administrator/Desktop/SNMOT-060"]
tst_dirs = ["C:/Users/Administrator/Desktop/SNMOT-116"]

split_dirs = {
    'train':trn_dirs,
    'valid':tst_dirs
}

# ４ types of labels are used.
labels = ['ball', 'player', 'referee', 'goalkeepers']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}

In [8]:
#(1) image file path
yolo_train_img_dir = f'{yolo_base}/images/train'
yolo_valid_img_dir = f'{yolo_base}/images/valid'

#(2) label file path
yolo_train_label_dir = f'{yolo_base}/labels/train'
yolo_valid_label_dir = f'{yolo_base}/labels/valid'

#(3) config file path
yaml_file = f'{yolo_base}/data.yaml'

#!rm -rf /home/tito/kaggle/dfl-bundesliga-data-shootout/work/yolov5
os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_valid_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_valid_label_dir, exist_ok=True)

In [9]:
# convert from x,y,w,h to yolo format
def get_yolo_format_bbox(img_w, img_h, box):
    w = box[2]
    h = box[3]
    xc = box[0] + int(np.round(w/2))
    yc = box[1] + int(np.round(h/2))
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box
    
# get SoccerNet label info 
def get_info(info):
    results = []
    for line in open(info):
        m = re.match('trackletID_(\d+)= (\S*).*', line.replace(';', ' '))
        if m:
            if m.group(2) not in label_dict:
                #print('bad label:', m.group(2))
                continue 
            results.append([m.group(1), m.group(2)])
    return pd.DataFrame(results, columns=['id','lbl']).set_index('id').to_dict()['lbl']

# make image simlink and label files
for split in split_dirs:
    if split == 'train':
        yolo_img_dir = yolo_train_img_dir
        yolo_label_dir = yolo_train_label_dir
    else:
        yolo_img_dir = yolo_valid_img_dir
        yolo_label_dir = yolo_valid_label_dir
        
    for this_dir in split_dirs[split]:
        print('this_dir:',this_dir)
        video = this_dir.split('/')[-1]
        info = this_dir + '/gameinfo.ini'
        det = this_dir + '/gt/gt.txt'
        info_dict = get_info(info)
        det_df = pd.read_csv(det, names=['frame','player','x','y','w','h','f1','f2','f3','f4'], usecols=['frame','player','x','y','w','h'])
        det_df['label'] = det_df.player.astype(str).map(info_dict)
        det_df['label_id'] = det_df['label'].map(label_dict)
        # check
        ng_list = list(det_df[det_df.label_id.isnull()].label.unique())
        if len(ng_list)>0:
            #print('ng_list:',ng_list, det_df.dropna().shape, det_df.shape)
            det_df = det_df.dropna()
        for grp, grp_df in det_df.groupby('frame'):
            frame = f'{grp:06}'
            img_file = f'{this_dir}/img1/{frame}.jpg'
            dst_file = f'{yolo_img_dir}/{video}_{frame}.jpg'
            if not os.path.exists(dst_file):
                os.symlink(img_file, dst_file)
                #print(img_file)
            img = cv2.imread(dst_file)
            height, width, _ = img.shape 
            bboxes = []
            for arr in grp_df[['x', 'y', 'w', 'h', 'label_id']].values:
                box = get_yolo_format_bbox(width, height, arr[:4])
                bboxes.append([arr[4]]+box)
            file_name = f'{yolo_label_dir}/{video}_{frame}.txt'
            with open(file_name, 'w') as f:
                for i, bbox in enumerate(bboxes):
                    bbox = [str(i) for i in bbox]
                    str_bbox = ' '.join(bbox)
                    f.write(str_bbox)
                    f.write('\n')

this_dir: C:/Users/Administrator/Desktop/SNMOT-060
this_dir: C:/Users/Administrator/Desktop/SNMOT-116


In [10]:
# Dump config file
data_yaml = dict(
    train = yolo_train_img_dir,
    val = yolo_valid_img_dir,
    nc = 4,
    names = labels
)

with open(yaml_file, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [13]:
# train!
weight_file = 'C:/Users/Administrator/Desktop/SoccerNet/yolov7/yolov7-w6_training.pt'
!cd $yolo_base && python train_aux.py \
    --workers 2 \
    --batch-size 4 \
    --data data.yaml \
    --img 1280 \
    --epochs 10 \
    --cfg cfg/training/yolov7-w6.yaml \
    --weights $weight_file \
    --name yolov7-w6-custom \
    --hyp data/hyp.scratch.custom.yaml

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)



YOLOR  2022-11-14 torch 1.11.0+cpu CPU

Namespace(weights='C:/Users/Administrator/Desktop/SoccerNet/yolov7/yolov7-w6_training.pt', cfg='cfg/training/yolov7-w6.yaml', data='data.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=10, batch_size=4, img_size=[1280, 1280], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=2, project='runs/train', entity=None, name='yolov7-w6-custom', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs\\train\\yolov7-w6-custom3', total_batch_size=4)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2022-11-28 15:22:29.677437: W tensorflow/stream_executor/platform/defaul

In [14]:
# predict!
check_images = f'test_imgs'
#weight_file = 'yolov7-w6_training.pt'
# weight_file = 'yolov7.pt'
!cd $yolo_base && python detect.py \
    --weights $weight_file \
    --source  $check_images \
    --img-size 1280 \
    --project dfl

Namespace(weights=['C:/Users/Administrator/Desktop/SoccerNet/yolov7/yolov7-w6_training.pt'], source='test_imgs', img_size=1280, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='dfl', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IAuxDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

16 persons, Done. (2329.7ms) Inference, (15.6ms) NMS
 The image with the result is saved in: dfl\exp\000001.jpg
16 persons, Done. (2200.8ms) Inference, (0.0ms) NMS
 The image with the result is saved in: dfl\exp\000002.jpg
17 persons, Done. (2218.3ms) Inference, (0.0ms) NMS
 The image with the result is saved in: dfl\exp\000003.jpg
17 persons, Done. (2219.0ms) Inference, (15.6ms) NMS
 The image with the result is saved in: dfl\exp\000004.jpg
17 persons, Done. (2218.2ms) Inference, (4.1ms) NMS
 The image with the result

YOLOR  2022-11-14 torch 1.11.0+cpu CPU

Model Summary: 370 layers, 82277300 parameters, 0 gradients
C:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# Check some results. 
display(Image(f'{yolo_base}/dfl/exp9/000200.jpg'))
display(Image(f'{yolo_base}/dfl/exp9/000400.jpg'))
display(Image(f'{yolo_base}/dfl/exp9/000600.jpg'))